In [2]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from IPython.display import display

from glob import glob


def parse_energy_file(file_path):
    metrics = {}
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                parts = line.split()
                if len(parts) < 2:
                    continue
                
                full_metric = parts[0]
                value = float(parts[-1])
                
                if "{" in full_metric and "}" in full_metric:
                    metric_name = full_metric.split("{")[0]
                    labels = full_metric.split("{")[1].split("}")[0]
                    
                    if "mode=" in labels:
                        mode = labels.split("mode=")[1].split(",")[0].strip('"')
                        key = f"{metric_name}_{mode}"
                    else:
                        key = metric_name
                else:
                    key = full_metric
                
                metrics[key] = value
            except (IndexError, ValueError) as e:
                print(f"Error parsing line '{line}': {str(e)}")
                continue
    return metrics


def calculate_energy_consumption(before_metrics, after_metrics):
    if not before_metrics or not after_metrics:
        return None

    total_joules_key = "kepler_container_joules_total_dynamic"

    dynamic_key = f'{total_joules_key}'
    if dynamic_key in after_metrics and dynamic_key in before_metrics:
        total_energy = after_metrics[dynamic_key] - before_metrics[dynamic_key]
    else:
        return None

    return total_energy if total_energy >= 0 else None
def calculate_cpu_usage_time(before_metrics, after_metrics):
    if not before_metrics or not after_metrics:
        return None

    cpu_usage_key = "kepler_container_bpf_cpu_time_ms_total"

    if cpu_usage_key in after_metrics and cpu_usage_key in before_metrics:
        total_cpu_time = after_metrics[cpu_usage_key] - before_metrics[cpu_usage_key]
    else:
        return None

    return total_cpu_time if total_cpu_time >= 0 else None

def process_test_directory(base_dir, test_name, requests, gateway=False):
    """Process all report directories for a specific test case."""
    energy_consumptions = []
    cpu_time_consumptions = []

    # Determine the path pattern based on whether it's gateway or not
    path_pattern = os.path.join(base_dir, str(requests), "report_*")

    # Process each report directory
    for report_dir in glob(path_pattern):
        try:
            before_file_gateway = None
            after_file_gateway = None
            before_metrics_gateway = None
            after_metrics_gateway = None
            energy_gateway = 0
            
            if gateway:
                before_file_gateway = os.path.join(report_dir, "before_gateway.txt")
                after_file_gateway = os.path.join(report_dir, "after_gateway.txt")
                
            before_file = os.path.join(report_dir, "before.txt")
            after_file = os.path.join(report_dir, "after.txt")

            if os.path.exists(before_file) and os.path.exists(after_file):
                if gateway and os.path.exists(before_file_gateway) and os.path.exists(after_file_gateway):
                    before_metrics_gateway = parse_energy_file(before_file_gateway)
                    after_metrics_gateway = parse_energy_file(after_file_gateway)
                    energy_gateway = calculate_energy_consumption(
                        before_metrics_gateway, after_metrics_gateway
                    )
                
                
                before_metrics = parse_energy_file(before_file)
                after_metrics = parse_energy_file(after_file)
                energy = calculate_energy_consumption(before_metrics, after_metrics)
                cpu_time = calculate_cpu_usage_time(before_metrics, after_metrics)
                if energy is not None:
                    energy_consumptions.append(energy + energy_gateway)
                if cpu_time is not None:
                    cpu_time_consumptions.append(cpu_time)
        except Exception as e:
            print(f"Error processing {report_dir}: {str(e)}")
            continue

    if not energy_consumptions:
        return None
    if not cpu_time_consumptions:
        return None
    
    # Pearson correlation coefficient
    if len(energy_consumptions) != len(cpu_time_consumptions):
        print(f"Warning: Mismatched lengths for energy and CPU time in {test_name} with {requests} requests.")
        return None
    res = stats.spearmanr(cpu_time_consumptions, energy_consumptions, alternative='greater')
    
    mean_value = np.mean(energy_consumptions)
    std_value = np.std(energy_consumptions)
    ste_value = std_value / np.sqrt(len(energy_consumptions))
    rsd = (std_value / mean_value) * 100 if mean_value != 0 else 0
    rse = (ste_value / mean_value) * 100 if mean_value != 0 else 0


    # Calculate statistics
    statistics = {
        "Test": test_name,
        "Requests": requests,
        "Mean": round(np.mean(energy_consumptions), 2),
        "Med": round(np.median(energy_consumptions), 2),
        "Min": round(np.min(energy_consumptions), 2),
        "Max": round(np.max(energy_consumptions), 2),
        "Std": round(np.std(energy_consumptions), 2),
        "RSD (%)": round(rsd, 2),
        "RSE (%)": round(rse, 2),
        "CPU Time (ms)": round(np.mean(cpu_time_consumptions), 2),
        "Monotonic relationship p-value":  round(res.pvalue, 6),
        "Monotonic relationship coefficient": round(res.correlation, 4),
        
        
    }

    return statistics


def get_endpoint_name(path):
    """Extract endpoint name from path"""
    parts = path.split("/")
    # Look for parts like 'json_get_user_id' or 'protobuf_get_all_users'
    for part in parts:
        if part.startswith("json_") or part.startswith("protobuf_"):
            return part.replace("json_", "").replace("protobuf_", "")
    return "unknown_endpoint"


def custom_sort(row):
    """Custom sorting function for the desired order"""
    test = row["Test"]
    requests = row["Requests"]

    if "Gateway" not in test:
        if requests == 1000:
            return 0 if "JSON" in test else 1
        else:
            return 2 if "JSON" in test else 3
    else:
        if requests == 1000:
            return 4 if "JSON" in test else 5
        else:
            return 6 if "JSON" in test else 7


test_dirs = [
    "../Control_Project_JSON/k6/json_get_all_products_entities/reports/json_get_all_products_entities",
    "../Control_Project_JSON/k6/json_get_user_id/reports/json_get_user_id",
    "../Control_Project_JSON/k6/json_get_all_users/reports/json_get_all_users",
    "../Control_Project_JSON/k6/json_create_user/reports/json_create_user",
    "../Control_Project_JSON/k6/json_update_user/reports/json_update_user",
    "../Control_Project_JSON/k6/json_delete_user/reports/json_delete_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_all_products_entities/reports/protobuf_get_all_products_entities",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_all_users/reports/protobuf_get_all_users",
    "../Experimental_Group_ProtoBuff/k6/protobuf_get_user_id/reports/protobuf_get_user_id",
    "../Experimental_Group_ProtoBuff/k6/protobuf_create_user/reports/protobuf_create_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_update_user/reports/protobuf_update_user",
    "../Experimental_Group_ProtoBuff/k6/protobuf_delete_user/reports/protobuf_delete_user",
    "../Gateway/k6_json/json_get_all_products_entities/reports/json_get_all_products_entities_gateway",
    "../Gateway/k6_json/json_get_all_users/reports/json_get_all_users_gateway",
    "../Gateway/k6_json/json_get_user_id/reports/json_get_user_id_gateway",
    "../Gateway/k6_json/json_create_user/reports/json_create_user_gateway",
    "../Gateway/k6_json/json_update_user/reports/json_update_user_gateway",
    "../Gateway/k6_json/json_delete_user/reports/json_delete_user_gateway",
    "../Gateway/k6_protobuf/protobuf_get_all_products_entities/reports/protobuf_get_all_products_entities_gateway",
    "../Gateway/k6_protobuf/protobuf_get_all_users/reports/protobuf_get_all_users_gateway",
    "../Gateway/k6_protobuf/protobuf_get_user_id/reports/protobuf_get_user_id_gateway",
    "../Gateway/k6_protobuf/protobuf_create_user/reports/protobuf_create_user_gateway",
    "../Gateway/k6_protobuf/protobuf_update_user/reports/protobuf_update_user_gateway",
    "../Gateway/k6_protobuf/protobuf_delete_user/reports/protobuf_delete_user_gateway",
]

# Group by endpoint
endpoint_data = {}

for test_dir in test_dirs:
    if not os.path.exists(test_dir):
        print(f"Directory not found: {test_dir}")
        continue

    # Determine endpoint name, test type, and whether it's gateway
    endpoint = get_endpoint_name(test_dir)
    is_gateway = "Gateway" in test_dir
    is_protobuf = "protobuf" in test_dir.lower()

    test_type = "PROTOBUF" if is_protobuf else "JSON"
    if is_gateway:
        test_type += " + Gateway"

    # Process for both request counts
    for requests in [1000, 10000]:
        statistics = process_test_directory(test_dir, test_type, requests, is_gateway)
        if statistics:
            if endpoint not in endpoint_data:
                endpoint_data[endpoint] = []
            endpoint_data[endpoint].append(statistics)

# Print results for each endpoint
for endpoint, results in endpoint_data.items():
    df = pd.DataFrame(results)

    # Add sorting key and sort
    df["sort_key"] = df.apply(custom_sort, axis=1)
    df = df.sort_values("sort_key").drop("sort_key", axis=1)

    df = df[
        [
            "Test",
            "Requests",
            "Mean",
            "Med",
            "Min",
            "Max",
            "Std",
            "RSD (%)",
            "RSE (%)",
            "CPU Time (ms)",
            "Monotonic relationship p-value",
            "Monotonic relationship coefficient",
        ]
    ]

    print(
        f"\nEnergy Consumption Statistics for {endpoint.replace('_', ' ').title()} (Joules):"
    )
    display(df)

    comparisons = []
    for (is_gateway, req_count), group in df.groupby([df["Test"].str.contains("Gateway"), "Requests"]):
        if len(group) == 2:  
            json_row = group[group["Test"].str.contains("JSON")].iloc[0]
            proto_row = group[group["Test"].str.contains("PROTOBUF")].iloc[0]

            test_name = "PROTOBUF + Gateway vs JSON + Gateway" if is_gateway else "PROTOBUF vs JSON"
            diff_pcts = {
                "Test": f"{test_name} ({req_count} requests)",
                "Mean %": (
                    (json_row["Mean"] - proto_row["Mean"]) / json_row["Mean"]
                )
                * 100,
                "Median %": ((json_row["Med"] - proto_row["Med"]) / json_row["Med"])
                * 100,
                "Min %": ((json_row["Min"] - proto_row["Min"]) / json_row["Min"])
                * 100,  
                "Max %": ((json_row["Max"] - proto_row["Max"]) / json_row["Max"])
                * 100,
                "Std %": ((json_row["Std"] - proto_row["Std"]) / json_row["Std"])
                * 100,
            }
            comparisons.append(diff_pcts)

    # Create comparison table
    comparison_df = pd.DataFrame(comparisons)
    comparison_df = comparison_df.round(2)  # Round to 2 decimal places

    print(f"\nPercentage Comparison for {endpoint.replace('_', ' ').title()}:")
    display(comparison_df)


Energy Consumption Statistics for Get All Products Entities (Joules):


,Test,Requests,Mean,Med,Min,Max,Std,RSD (%),RSE (%),CPU Time (ms),Monotonic relationship p-value,Monotonic relationship coefficient
0,JSON,1000,22.91,15.23,4.42,97.59,26.17,114.22,36.12,3566.9,0.000000,1.0000
2,PROTOBUF,1000,18.66,18.03,3.64,58.89,14.52,77.83,24.61,2924.6,0.000000,0.9879
1,JSON,10000,80.19,81.05,73.85,84.59,3.68,4.58,1.45,10854.7,0.000000,0.9879
3,PROTOBUF,10000,73.12,72.82,69.49,80.28,3.11,4.25,1.34,9825.8,0.000027,0.9394
4,JSON + Gateway,1000,51.29,40.06,8.19,178.66,44.67,87.10,27.54,3850.3,0.000000,1.0000
6,PROTOBUF + Gateway,1000,31.78,28.89,5.61,72.88,17.34,54.57,17.26,3248.3,0.000001,0.9758
5,JSON + Gateway,10000,216.30,215.75,203.97,232.72,9.38,4.34,1.37,13451.1,0.000000,0.9879
7,PROTOBUF + Gateway,10000,204.99,208.63,192.83,211.87,6.74,3.29,1.04,12246.5,0.000000,1.0000



Percentage Comparison for Get All Products Entities:


,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),18.55,-18.38,17.65,39.66,44.52
1,PROTOBUF vs JSON (10000 requests),8.82,10.15,5.90,5.10,15.49
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,38.04,27.88,31.50,59.21,61.18
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,5.23,3.30,5.46,8.96,28.14



Energy Consumption Statistics for Get User Id (Joules):


...

,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),-4.09,-4.44,-9.23,-0.38,50.13
1,PROTOBUF vs JSON (10000 requests),-6.39,-7.03,-4.46,-6.90,-57.59
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,-0.29,-1.85,-3.08,3.39,39.03
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,-3.65,-3.55,-3.32,-3.45,-11.48



Energy Consumption Statistics for Delete User (Joules):


,Test,Requests,Mean,Med,Min,Max,Std,RSD (%),RSE (%),CPU Time (ms),Monotonic relationship p-value,Monotonic relationship coefficient
0,JSON,1000,66.71,65.31,59.62,74.98,5.56,8.34,2.64,11939.8,0.0,0.9879
2,PROTOBUF,1000,66.65,65.29,57.30,80.85,6.23,9.35,2.96,11686.0,0.0,0.9879
1,JSON,10000,364.71,345.18,85.83,660.38,178.45,48.93,15.47,62014.3,0.0,1.0000
3,PROTOBUF,10000,376.93,355.23,96.86,664.42,185.64,49.25,15.57,61522.4,0.0,1.0000
4,JSON + Gateway,1000,80.06,80.40,72.03,89.56,5.28,6.60,2.09,11959.9,0.0,0.9879
6,PROTOBUF + Gateway,1000,83.07,84.11,74.64,93.03,5.26,6.33,2.00,12373.0,0.0,0.9879
5,JSON + Gateway,10000,524.59,518.80,248.75,804.02,169.64,32.34,10.23,65430.5,0.0,1.0000
7,PROTOBUF + Gateway,10000,537.43,525.56,263.54,808.43,165.38,30.77,9.73,66685.7,0.0,1.0000



Percentage Comparison for Delete User:


,Test,Mean %,Median %,Min %,Max %,Std %
0,PROTOBUF vs JSON (1000 requests),0.09,0.03,3.89,-7.83,-12.05
1,PROTOBUF vs JSON (10000 requests),-3.35,-2.91,-12.85,-0.61,-4.03
2,PROTOBUF + Gateway vs JSON + Gateway (1000 req...,-3.76,-4.61,-3.62,-3.87,0.38
3,PROTOBUF + Gateway vs JSON + Gateway (10000 re...,-2.45,-1.30,-5.95,-0.55,2.51
